In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import torchvision
import torchvision.transforms as transforms

In [11]:
transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [12]:
batch_size = 32
num_workers = 2
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
image, label = train_data[0]
image.size()

torch.Size([3, 32, 32])

In [14]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [15]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) #32 - 5 + 1 = 28 -> 28x28x12
        self.pool = nn.MaxPool2d(2, 2) # 28 / 2 = 14 -> 14x14x12
        self.conv2 = nn.Conv2d(12, 24, 5) # 14 - 5 + 1 = 10 -> 10x10x24
        #self.poll again                 10 / 2 = 5 -> 5x5x24
        self.Linear1 = nn.Linear(24*5*5, 120) # 24*5*5 = 600 -> 120
        self.Linear2 = nn.Linear(120, 60) # 120 -> 80
        self.Linear3 = nn.Linear(60, 10) # 80 -> 10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.Linear1(x)) 
        x = F.relu(self.Linear2(x))
        x = self.Linear3(x)
        return x

In [16]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)

net.apply(weights_init)

NeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (Linear1): Linear(in_features=600, out_features=120, bias=True)
  (Linear2): Linear(in_features=120, out_features=60, bias=True)
  (Linear3): Linear(in_features=60, out_features=10, bias=True)
)

In [17]:
for epoch in range(30):
    print(f'Epoch {epoch + 1}...')

    running_loss = 0.0
    net.train()

    for i,data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

Epoch 1...
Epoch 2...
Epoch 3...
Epoch 4...
Epoch 5...
Epoch 6...
Epoch 7...
Epoch 8...
Epoch 9...
Epoch 10...
Epoch 11...
Epoch 12...
Epoch 13...
Epoch 14...
Epoch 15...
Epoch 16...
Epoch 17...
Epoch 18...
Epoch 19...
Epoch 20...
Epoch 21...
Epoch 22...
Epoch 23...
Epoch 24...
Epoch 25...
Epoch 26...
Epoch 27...
Epoch 28...
Epoch 29...
Epoch 30...


In [18]:
avg_loss = running_loss / len(train_loader)
print(f'Loss: {avg_loss:.3f}')

Loss: 0.779


In [19]:
torch.save(net.state_dict(), 'model.pth')
net = NeuralNet()
net.load_state_dict(torch.load('model.pth'))

/var/folders/tm/ntqqt3b165j765q0kw8xndkh0000gn/T/ipykernel_97527/3914640100.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [20]:
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct*100 / total
print(f'Accuracy: {accuracy}%')

Accuracy: 69.28%


In [21]:
import torchvision.transforms as transforms
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image).unsqueeze(0)
    return image

image_paths = ['Ex1.jpeg', 'Ex2.jpeg', 'Ex3.jpeg']
images = [load_image(image_path) for image_path in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        outputs = net(image)
        _, predicted = torch.max(outputs, 1)
        print(f'Predicted: {class_names[predicted.item()]}')

Predicted: deer
Predicted: plane
Predicted: cat
